In [1]:
import os
import datetime
# Local Imports
import coastseg
from coastseg import coastseg_map
from coastseg import coastseg_logs #must be the first module loaded to create logs folder

# External Imports
import ee
from google.auth import exceptions as google_auth_exceptions
from IPython.display import display, clear_output
from tkinter import Tk,filedialog
from ipywidgets import Button, HBox, VBox, Layout, DatePicker, HTML, RadioButtons, SelectMultiple, Output, Checkbox

# suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

## Authenticate and Initialize with Google Earth Engine (GEE)

- Run this cell to initialize with GEE which will allow you to download remote sensing data from GEE.

### First Time Users

- In order to use Google Earth Engine (GEE) you will need to sign up to request access to use Google Earth Engine.https://signup.earthengine.google.com. You will only need to do this once and it takes only a day to get your account verified.

### How `ee.Authenticate()` works

- In order to initialize with GEE you will need an authorization token with is obtained by running `ee.Authenticate()`.This token lasts 7 days and during those 7 days you will not need to authenticate with google earth engine with an access code. Once the 7 days are up you will need to reauthenticate to use GEE again.


In [2]:
try:
    ee.Initialize()
except google_auth_exceptions.RefreshError as exception:
    print("Please authenticate with Google:\n")
    ee.Authenticate()
    ee.Initialize()

In [3]:
from ipywidgets import Output
from ipywidgets import Layout, HTML, Accordion
from coastseg.tkinter_window_creator import Tkinter_Window_Creator

debug_view = Output(layout={"border": "1px solid black"})

@debug_view.capture(clear_output=True)
def load_feature_from_file(btn):
    filename = None
    with Tkinter_Window_Creator() as tk_root:
            tk_root.filename = filedialog.askopenfilename(
                initialdir=os.getcwd(),
                filetypes=[("geojson", "*.geojson")],
                title="Select a geojson file containing rois",
            )
    if tk_root.filename:    
        filename = tk_root.filename
    else:
        messagebox.showerror(
            "File Selection Error", "You must select a valid geojson file first!"
        )
    # if a valid file was loaded call corresponding loading function
    if filename: 
        if 'shoreline' in btn.description.lower():
            print(f"Loading Shoreline from file: {filename}")
        if 'transects' in btn.description.lower():
            print("Calling Transects")

In [4]:
from_file_instr=HTML(
    value="<h3>Load Feature From File</h3>",
    layout=Layout(padding='0px')
)

from_file_radio=RadioButtons(
    options=['Shoreline', 'Transects','Bbox','ROIs',],
    value='Shoreline',
    description='',
    disabled=False
)


from_file_button = Button(
    description=f"Load {from_file_radio.value} file",
    )
from_file_button.on_click(load_feature_from_file)

def handle_from_file_change(change):
    from_file_button.description=f"Load {str(change['new'])} file"


from_file_vbox=VBox([from_file_instr,from_file_radio,from_file_button,debug_view])
from_file_radio.observe(handle_from_file_change,"value")
from_file_vbox

In [5]:
from ipywidgets import Output
from ipywidgets import Layout, HTML, Accordion

debug_view = Output(layout={"border": "1px solid black"})

@debug_view.capture(clear_output=True)
def save_feature_to_file(btn):
    if 'shoreline' in btn.description.lower():
        print("Calling Shoreline")
    if 'transects' in btn.description.lower():
        print("Calling Transects")

In [6]:

save_instr=HTML(
    value="<h3>Save to file</h3>",
    layout=Layout(padding='0px')
)

save_radio=RadioButtons(
    options=['Shoreline', 'Transects','Bbox','ROIs',],
    value='Shoreline',
    description='',
    disabled=False
)


save_button = Button(
    description=f"Save {save_radio.value} to file",
    )
save_button.on_click(save_feature_to_file)

def handle_save_change(change):
    save_button.description=f"Save {str(change['new'])} to file"


save_vbox=VBox([save_instr,save_radio,save_button,debug_view])
save_radio.observe(handle_save_change,"value")
save_vbox

In [7]:
from ipywidgets import Output
from ipywidgets import Layout, HTML, Accordion

debug_view = Output(layout={"border": "1px solid black"})

@debug_view.capture(clear_output=True)
def f(btn):
    if 'shoreline' in btn.description.lower():
        print("Calling Shoreline")
        coastsegmap.load_feature_on_map('shoreline')
    if 'transects' in btn.description.lower():
        print("Calling Transects")
    

In [8]:

instr=HTML(
    value="<h3>Load on Map</h3>",
    layout=Layout(padding='0px')
)

load_radio=RadioButtons(
    options=['Shoreline', 'Transects'],
    value='Shoreline',
    description='',
    disabled=False
)


load_button = Button(
    description=f"Load {load_radio.value}",
    )
load_button.on_click(f)

def handle_collection_change(change):
    load_button.description=f"Load {str(change['new'])}"


collection_vbox=VBox([instr,load_radio,load_button,debug_view])
load_radio.observe(handle_collection_change,"value")
collection_vbox

In [23]:
coastsegmap.shoreline.gdf.to_file("shoreline_sample.geojson",driver="GeoJSON")

In [9]:
settings = { 
    # general parameters:
    'cloud_thresh': 0.5,        # threshold on maximum cloud cover
    'dist_clouds': 300,        # ditance around clouds where shoreline can't be mapped
    'output_epsg': 3857,        # epsg code of spatial reference system desired for the output   
    # quality control:
    'check_detection': False,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 550,         # radius (in metres) of the buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 100,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    'sand_color': 'default',    # 'default', 'dark' (for grey/black sand beaches) or 'bright' (for white sand beaches)
    'pan_off':'False',          # if True, no pan-sharpening is performed on Landsat 7,8 and 9 imagery
    'create_plot':False,        # True create a matplotlib plot of the image with the datetime as the title
    'max_dist_ref':25,
    'along_dist' :25,
}

In [16]:
coastsegmap=coastseg_map.CoastSeg_Map()

In [11]:
# Date Widgets
start_date=DatePicker(
    description='Start Date',
    value=datetime.date(2018, 12, 1),
    disabled=False,
)
end_date=DatePicker(
    description='End Date',
    value=datetime.date(2019, 3, 1), #2019, 1, 1
    disabled=False,
)
date_instr=HTML(
    value="<b>Pick a date:</b>",
    layout=Layout(padding='10px')
)
dates_box=HBox([start_date,end_date])
dates_vbox=VBox([date_instr,dates_box])

sat_instr=HTML(
    value="<b>Pick multiple satellites by holding the control key:</b> \
        <br> - images after 2022/01/01 will be automatically downloaded from Collection 2 \
        <br> C02: LandSat Collection 2 : All dates after 2022/01/01 \
        <br> - Landsat 9 is only available in C02",
    layout=Layout(padding='10px')
)


satellite_selection=SelectMultiple(
    options=['L5', 'L7', 'L8', 'S2'],
    value=['L8'],
    description='Satellites',
    disabled=False
)
sat_vbox = VBox([sat_instr,satellite_selection])

from ipywidgets import Layout, HTML, Accordion



@debug_view.capture(clear_output=True)
def save_settings_clicked(btn):
    if satellite_selection.value:
        sat_list = list(satellite_selection.value)
    elif not satellite_selection.value:
        raise Exception("Must select at least one satellite first")
    # Save dates selected by user
    dates = [str(start_date.value),str(end_date.value)]
    print(f"dates: {dates}")
    print(f"sat_list: {sat_list}")
    coastsegmap.save_settings(sat_list=sat_list, landsat_collection = 'C02',dates=dates,**settings)

settings_button = Button(
    description="Save Settings",
    )
settings_button.on_click(save_settings_clicked)

controls_vbox=VBox([dates_vbox, sat_vbox,settings_button,debug_view])
# controls_vbox
accordion = Accordion(children=[controls_vbox])
accordion.set_title(0, "Settings")
accordion

Accordion(children=(VBox(children=(VBox(children=(HTML(value='<b>Pick a date:</b>', layout=Layout(padding='10p…

In [12]:
# CoastSat Download Variables
# ------------------------------
# Save the satellites selected to sat_list
if satellite_selection.value:
    sat_list = list(satellite_selection.value)
elif not satellite_selection.value:
    print("ERROR: You must select at least one satellite first")
# Save the dates selected by the user as well as the selected collection
dates = [str(start_date.value),str(end_date.value)]
print(f"dates: {dates}")
print(f"sat_list: {sat_list}")

dates: ['2018-12-01', '2019-03-01']
sat_list: ['L8']


## Settings for CoastSat

Modify any of the following setting to determine how data is downloaded from CoastSat
## Future Plan
 All of the settings and the widgets to control the settings will be incorperated into the main dashboard

# Create the Map

- Create the map here
- Save all the settings created earlier
- Save the list of satellites, dates, and the collection to coastseg_map 


## Future Plan
Note: This save settings function will eventually be converted into a button.


In [13]:
# coastsegmap=coastseg_map.CoastSeg_Map()

In [14]:
# coastsegmap.save_settings(sat_list=sat_list, landsat_collection = 'C02',dates=dates,**settings)

In [18]:
coastsegmap.settings

{'sat_list': ['L8'],
 'landsat_collection': 'C02',
 'dates': ['2018-12-01', '2019-03-01'],
 'cloud_thresh': 0.5,
 'dist_clouds': 300,
 'output_epsg': 3857,
 'check_detection': False,
 'adjust_detection': False,
 'save_figure': True,
 'min_beach_area': 4500,
 'buffer_size': 550,
 'min_length_sl': 100,
 'cloud_mask_issue': False,
 'sand_color': 'default',
 'pan_off': 'False',
 'create_plot': False,
 'max_dist_ref': 25,
 'along_dist': 25}

# How to Use The Map

---

1. Use the rectangle tool to draw a bounding box along the coastline.
2. Click `Generate ROI` to create ROI rectangles along the coastline in the bounding box. This may take some time.
- You should see a coastline appear in yellow and some rectangles along it.
3. Load transects into your bounding box with the `Load Transects` button. If any exist for the bounding box you selected they should appear.
4. Click the ROIs you want to download.
5. Once you've selected all the ROIs you want to download click `Downlod ROIs`
 - If any of the ROIs succesfully download they will have their own folder with all their data in the `data` directory in the `CoastSeg` directory
6. To extract a timeseries of shorelines for the ROIs you downloaded click `Extract Shorelines`
- Note this feature is still under development. This instruction will be updated later
7. If any shorelines were extracted you can compute the cross distances along the transects and shoreline by clicking `Compute Transects` 
- Note this feature is still under development. This instruction will be updated later

## How to Use Config Files
---
Config files are used to save and restore the state of your map. When you click `save config` or `download ROI` all the selected ROIs, transects, shorelines and the current bounding box loaded onto the map are saved to two config files: `config.json` and `config_gdf.geojson`. Extracted shorelines and cross distances for transects are not saved to the config files.

## Saving Config Files
---
Config files are automatically saved in the `data` directory within each ROI directory when `Download ROIs` is clicked. 
Config files can be saved at any time when the `Save Config Files` button. If the ROIs selected on the map have been downloaded before and exist in the `data` directory then the config files will be stored in  within each ROI directory. If the ROIs on the map have not been downloaded before then the config files will be saved in the coastseg directory.

#### Steps
1. Click the ROIs on the map you want to save
2. Click the `Save Config Files` button. 
- If the ROIs have been downloaded the config file is in each ROI's folder in file: 'config_gdf.geojson'
- If the ROIs have not been downloaded the config file is in main CoastSeg directory in file: 'config_gdf.geojson'
- The 'config.json' will be saved in the same directory as the 'config_gdf.geojson'.

## Loading Config Files
___
### Load Config Files For Download ROIs
To load config files for ROIs that have been downloaded before go to the `data` directory within `coastseg` then find the directory of the ROIs you want to upload. Inside that directory, for example the 
`ID_3_datetime10-20-22__07_09_07` directory, there should be `geojson` file named `config_gdf_id_3.geojson`. You should notice the `ID_<ROI ID NUMBER>` in the directory name and the geojson file match, in this case this is ROI id 3. Upload the `config_gdf_id_<roi id number>.geojson`.
This will load the ROI in that directory and all the other selected ROIs,shorelines, transects, and the bounding box that were on the map when it was saved.
#### Steps
1. Click `Load Config`
2. Open `data` directory
3. Open ROI directory ex. **ID_3_datetime10-20-22__07_09_07**
3. Select `config_gdf_id_<roi id number>.geojson` ex. **config_gdf_id_3.geojson**

### Load Config Files For ROIs That Have Not Been Downloaded
To load config files for ROIs that have not been downloaded before first click the `Load Config` button,  open the coastseg directory and select `config_gdf.geojson`. This will loaded the selected ROIs, shorelines, transects, and bounding box that was saved with the `Save Config Files` button. 
NOTE: You will not be able to extract shorelines or compute transects until you download the data for these ROIs.
#### Steps
1. Click `Load Config`
2. Open `coastseg` application directory
3. Select `config_gdf.geojson`

In [19]:
from src.coastseg.map_UI import UI
coastseg_ui = UI(coastsegmap)
coastseg_ui.create_dashboard()

# The Rest of this Notebook is Still Under Active Development
---

# Extract Shorelines From Downloaded Imagery
------
Extracted Shorelines are shoreline vectors in the imagery you downloaded from GEE. The extracted shorelines are created by analyzing each `.tiff` file for each set of satellite imagery downloaded to check if a shoreline classifier can be applied to it. If the `.tiff` file meets the quality standards by being under the maxiumum theshold for cloud cover and other metrics the image is analyzed by a special image segmentation model which classifies the sand, water and surf in the image. Using the labeled images from the image segmentation model a shoreline vector will be detected in the image and saved to a dictionary entry for the ROI it belonged to. The extracted shoreline will appear in purple on the map.

## Beta Shorelines Do Not Always Work Properly
The classifier to extract shorelines is still under development and fails for complex shorelines sometimes. If you see extracted shorelines that zig and zag this is likely the result of the classifier failing to classifiy your image. You may want to try changing the sand color parameter of the classifier if the sand on your coast is dark or light.

## Current Bugs Under Development
1. Multiple shoreline vectors will be rendered onto the map at the same location because all the shorelines accross the time period you selected are calculated at the same time. In the next release of CoastSeg these will be rendered one at a time on the map and you will be able to select which dates are rendered.
2. Sometimes shorelines will be not be able to be extracted from any of the imagery downloaded for the ROI on the map due to image quality issues such as excessive cloud cover or too many 'no data'(black) pixels in the downloaded imagery. When this is the case the extracted shoreline will not appear on the map. In a future update of coastseg there will be error messages to tell you when this has occured.


## How to Extract Shorelines From Your Iamgery
See the extracted shorelines for the ROIs selected on the map by first selecting your ROIs, downloading their data from GEE and then clicking extract shorelines.
1. Click the ROIs to extract shorelines from on the map
2. Download the ROIs data from GEE or already have it downloaded.
3. Click `extract shorelines`
4. You should see extracted shorelines appear in purple within some of your ROIs.
5. Run the following code block to see the extracted shorelines for each ROI.

In [ ]:
if coastsegmap.rois is None or len(coastsegmap.selected_set) ==0:
    print("Please select ROIs on the map first")
elif coastsegmap.rois is not None:
    if coastsegmap.data_downloaded == False:
        print("Download the data first to extract shorelines from it")
    elif coastsegmap.data_downloaded == True:
        print(coastsegmap.rois.extracted_shorelines)

# Compute Transect Cross Distance for Extracted Shorelines
--------
# How Compute Transect Cross Distance for Extracted Shorelines
1. Click `Extract Shorelines`
2.  Click `Compute Transects`

## Run Models on Imagery
1. Select if you want to use a GPU or not. By default you won't your computer's GPU
2. Select if you want to use CRF post-processing or not. By default CRF post-processing is not enabled
<br>**WARNING**: There is a bug with CRF post-processing and the models so you will need to enable CRF post-processing until it is fixed.
3. Select whether your data is the type RGB or MNDWI
4. Select the model you want to use
5. Select Ensemble or Best (Ensemble is recommended) 
6. 


In [ ]:
from src.coastseg.UI_models import UI_Models
models_ui = UI_Models()
models_ui.create_dashboard()